In [ ]:
# import all the libraries
import numpy as np
import argparse
import time
from random import shuffle
import random
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Flatten, Input, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.regularizers import l1,l2
from tensorflow.keras.optimizers import Adam

In [ ]:
# connect to google drive
from google.colab import drive
drive.mount('/content/myDrive/')

Mounted at /content/myDrive/


In [ ]:
# go to folder with /data and /utils
cd myDrive/MyDrive/ZSL

/content/myDrive/MyDrive/ZSL


In [ ]:
# set device as GPU
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Zero shot learning - final project
## DL for Spiking Neural Networks and Advanced Data Mining

In [ ]:
from scipy import io
att_splits=io.loadmat('./data/AWA2/binaryAtt_splits.mat')
# retrieve all the class names
classes_names = [a[0] for a in np.squeeze(att_splits['allclasses_names'])]
# create helper dictionary from class id to class name
id_to_name = {i:name for (i,name) in enumerate(classes_names)}

### Reading data

In [ ]:
from utils import data_loader
import importlib
# load /utils/data_loader
importlib.reload(data_loader)
# load the data
trainDataX, trainDataLabels, trainDataAttrs, testDataX, testDataLabels, testDataAttrs = data_loader.DataLoader("./data/CUB", binary=False).load()
trainDataX = (trainDataX - np.mean(trainDataX)) / np.std(trainDataX) # normalize image embeddings
testDataX = (testDataX - np.mean(testDataX)) / np.std(testDataX) # normalize image embeddings

# get the number of testing examples
test_size = len(testDataLabels)

In [ ]:
# check total number of classes (test & train)
num_classes = len(np.unique(np.concatenate((trainDataLabels, testDataLabels))))
num_classes

50

### Prepare input shape

In [ ]:
# prepare the input tensor for the model
in_shape = trainDataX.shape[1]
inputs = Input(shape=(in_shape,))



### Define useful for evaluation dictionaries

In [ ]:
# create class_id:attribute dictionary
class_attr_dict = {}
for A, B in zip(trainDataLabels, trainDataAttrs):
    # training classes
    class_attr_dict[A] = B
for A, B in zip(testDataLabels, testDataAttrs):
    # testing classes
    class_attr_dict[A] = B

# sort it from smallest to highest id
keys = sorted(class_attr_dict)
class_attr_dict = {i: class_attr_dict[i] for i in keys}
class_attr_dict.keys()

# create dictionary from attributes to a class id
atrr_to_class = {v.tobytes(): k for k, v in class_attr_dict.items()}
all_class_atrs = class_attr_dict.values()

In [ ]:
# create class_id:attribute dictionary for test classes
test_attributes = {}
for A, B in zip(testDataLabels, testDataAttrs):
    test_attributes[A] = B
test_attributes = test_attributes.values()

In [ ]:
def difference(class_attribute, prediction):
    ''' calulate difference between predicted and actual attribute values '''
    # assert(len(class_attribute) == len(prediction))
    dif = 0
    # for the each singular attribute in actual attribute representation
    for i, att in enumerate(class_attribute):
        # add the difference between the prediction attribute and class attribute
        dif += abs(prediction[i] - att)
    # return sum of differences
    return dif

def calculate_accuracy(pred, printing=False):
  ''' get accuracy of the given model given its prediction of test dataset '''
  correct = 0

    # iterate through each test example
  for id in range(test_size-1):
    # get distance score
    differences = [difference(att, pred[id]) for att in test_attributes]
    #assert(len(differences)==10)

    # take the id of the min distance
    id_predicted = np.argmin(differences)
    # retrive the class id of the min distance
    class_prediction = atrr_to_class[list(test_attributes)[id_predicted].tobytes()]

    # optional printing
    if printing:
      print(f"predicted class: {class_prediction} real class : {testDataLabels[id]}")
      print(f"predicted class: {id_to_name[class_prediction]} real class : {id_to_name[testDataLabels[id]]}")
      print(f"CORRECT:::??? ->>> {class_prediction == testDataLabels[id]}")

    # if correct, add to correct variable
    if class_prediction == testDataLabels[id]:
      correct += 1

  # return accuracy
  return correct/test_size

def get_prediction(model):
  ''' get prediction of test dataset of a given model '''
  # get prediction on test set
  pred = model.predict(testDataX)

  # transform to fit the attributes shape and form
  pred = list(np.array(pred).T)[0]

  # return prediction in form of [attA probability, attB probability, ...] of size 85
  return pred

## Prepare the models

In [ ]:
x = Dropout(0.8)(inputs) # 0.8 dropout
# linear activation for regression
# 85 neurons as 85 attributes in AWA2
x = Dense(85, activation='linear')(x)

model1 = Model(inputs, x)

with tf.device(device_name):
  model1.compile(optimizer=Adam(learning_rate=0.001), loss='mae', metrics=['mae'])

hist1 = model1.fit(trainDataX, trainDataAttrs, batch_size=64, shuffle=True, epochs=10, callbacks=[EarlyStopping(monitor='loss',min_delta=0.0001, patience=2)])

Epoch 1/10
460/460 [==============================] - 2s 3ms/step - loss: 0.7952 - mae: 0.7952
Epoch 2/10
460/460 [==============================] - 1s 3ms/step - loss: 0.2550 - mae: 0.2550
Epoch 3/10
460/460 [==============================] - 1s 3ms/step - loss: 0.2542 - mae: 0.2542
Epoch 4/10
460/460 [==============================] - 2s 4ms/step - loss: 0.2537 - mae: 0.2537
Epoch 5/10
460/460 [==============================] - 2s 4ms/step - loss: 0.2542 - mae: 0.2542
Epoch 6/10
460/460 [==============================] - 1s 3ms/step - loss: 0.2533 - mae: 0.2533
Epoch 7/10
460/460 [==============================] - 1s 3ms/step - loss: 0.2527 - mae: 0.2527
Epoch 8/10
460/460 [==============================] - 1s 3ms/step - loss: 0.2539 - mae: 0.2539
Epoch 9/10
460/460 [==============================] - 1s 3ms/step - loss: 0.2531 - mae: 0.2531


In [ ]:
x = Dropout(0.8)(inputs) # 0.8 dropout
# linear activation for regression
# 85 neurons as 85 attributes in AWA2
x = Dense(85, activation='linear')(x)

model2 = Model(inputs, x)

with tf.device(device_name):
  model2.compile(optimizer=Adam(learning_rate=0.0005), loss='mae', metrics=['mae'])

hist2 = model2.fit(trainDataX, trainDataAttrs, batch_size=64, shuffle=True, epochs=10, callbacks=[EarlyStopping(monitor='loss',min_delta=0.0001, patience=2)])

Epoch 1/10
460/460 [==============================] - 2s 3ms/step - loss: 1.1746 - mae: 1.1746
Epoch 2/10
460/460 [==============================] - 1s 3ms/step - loss: 0.1697 - mae: 0.1697
Epoch 3/10
460/460 [==============================] - 1s 3ms/step - loss: 0.1334 - mae: 0.1334
Epoch 4/10
460/460 [==============================] - 1s 3ms/step - loss: 0.1328 - mae: 0.1328
Epoch 5/10
460/460 [==============================] - 1s 3ms/step - loss: 0.1323 - mae: 0.1323
Epoch 6/10
460/460 [==============================] - 1s 3ms/step - loss: 0.1323 - mae: 0.1323
Epoch 7/10
460/460 [==============================] - 2s 3ms/step - loss: 0.1322 - mae: 0.1322


In [ ]:
x = Dropout(0.8)(inputs) # 0.8 dropout
# linear activation for regression
# 85 neurons as 85 attributes in AWA2
x = Dense(85, activation='linear')(x)

model3 = Model(inputs, x)

with tf.device(device_name):
  model3.compile(optimizer=Adam(learning_rate=0.0001), loss='mae', metrics=['mae'])

hist3 = model3.fit(trainDataX, trainDataAttrs, batch_size=64, shuffle=True, epochs=10, callbacks=[EarlyStopping(monitor='loss',min_delta=0.0001, patience=2)])

Epoch 1/10
460/460 [==============================] - 2s 3ms/step - loss: 2.0331 - mae: 2.0331
Epoch 2/10
460/460 [==============================] - 1s 3ms/step - loss: 1.4597 - mae: 1.4597
Epoch 3/10
460/460 [==============================] - 1s 3ms/step - loss: 1.0041 - mae: 1.0041
Epoch 4/10
460/460 [==============================] - 2s 4ms/step - loss: 0.6300 - mae: 0.6300
Epoch 5/10
460/460 [==============================] - 2s 4ms/step - loss: 0.3238 - mae: 0.3238
Epoch 6/10
460/460 [==============================] - 1s 3ms/step - loss: 0.1010 - mae: 0.1010
Epoch 7/10
460/460 [==============================] - 1s 3ms/step - loss: 0.0457 - mae: 0.0457
Epoch 8/10
460/460 [==============================] - 1s 3ms/step - loss: 0.0450 - mae: 0.0450
Epoch 9/10
460/460 [==============================] - 1s 3ms/step - loss: 0.0447 - mae: 0.0447
Epoch 10/10
460/460 [==============================] - 1s 3ms/step - loss: 0.0445 - mae: 0.0445


In [ ]:
x = Dropout(0.7)(inputs) # 0.7 dropout
# linear activation for regression
# 85 neurons as 85 attributes in AWA2
x = Dense(85, activation='linear')(x)

model4 = Model(inputs, x)

with tf.device(device_name):
  model4.compile(optimizer=Adam(learning_rate=0.0005), loss='mae', metrics=['mae'])

hist4 = model4.fit(trainDataX, trainDataAttrs, batch_size=64, shuffle=True, epochs=10, callbacks=[EarlyStopping(monitor='loss',min_delta=0.0001, patience=2)])

Epoch 1/10
460/460 [==============================] - 3s 3ms/step - loss: 0.9487 - mae: 0.9487
Epoch 2/10
460/460 [==============================] - 1s 3ms/step - loss: 0.1485 - mae: 0.1485
Epoch 3/10
460/460 [==============================] - 1s 3ms/step - loss: 0.1096 - mae: 0.1096
Epoch 4/10
460/460 [==============================] - 1s 3ms/step - loss: 0.1088 - mae: 0.1088
Epoch 5/10
460/460 [==============================] - 2s 5ms/step - loss: 0.1088 - mae: 0.1088
Epoch 6/10
460/460 [==============================] - 2s 4ms/step - loss: 0.1082 - mae: 0.1082
Epoch 7/10
460/460 [==============================] - 1s 3ms/step - loss: 0.1080 - mae: 0.1080
Epoch 8/10
460/460 [==============================] - 2s 4ms/step - loss: 0.1083 - mae: 0.1083
Epoch 9/10
460/460 [==============================] - 2s 4ms/step - loss: 0.1078 - mae: 0.1078
Epoch 10/10
460/460 [==============================] - 1s 3ms/step - loss: 0.1078 - mae: 0.1078


In [ ]:
x = Dropout(0.4)(inputs) # 0.4 dropout
# linear activation for regression
# 85 neurons as 85 attributes in AWA2
x = Dense(85, activation='linear')(x)

model5 = Model(inputs, x)

with tf.device(device_name):
  model5.compile(optimizer=Adam(learning_rate=0.0005), loss='mae', metrics=['mae'])

hist5 = model5.fit(trainDataX, trainDataAttrs, batch_size=64, shuffle=True, epochs=10, callbacks=[EarlyStopping(monitor='loss',min_delta=0.0001, patience=2)])

Epoch 1/10
460/460 [==============================] - 2s 3ms/step - loss: 0.6456 - mae: 0.6456
Epoch 2/10
460/460 [==============================] - 1s 3ms/step - loss: 0.1428 - mae: 0.1428
Epoch 3/10
460/460 [==============================] - 1s 3ms/step - loss: 0.0771 - mae: 0.0771
Epoch 4/10
460/460 [==============================] - 1s 3ms/step - loss: 0.0759 - mae: 0.0759
Epoch 5/10
460/460 [==============================] - 1s 3ms/step - loss: 0.0751 - mae: 0.0751
Epoch 6/10
460/460 [==============================] - 1s 3ms/step - loss: 0.0746 - mae: 0.0746
Epoch 7/10
460/460 [==============================] - 1s 3ms/step - loss: 0.0742 - mae: 0.0742
Epoch 8/10
460/460 [==============================] - 1s 3ms/step - loss: 0.0744 - mae: 0.0744
Epoch 9/10
460/460 [==============================] - 2s 4ms/step - loss: 0.0743 - mae: 0.0743


In [ ]:
models = [model1, model2, model3, model4, model5] # gather all the models
# evaluate each model on the accuracy on test set
accuracies = [calculate_accuracy(m.predict(testDataX), printing=False) for m in models]
accuracies

248/248 [==============================] - 1s 2ms/step


[0.3294578541640339,
 0.4733982054846455,
 0.3815240743081006,
 0.3273094907114874,
 0.3661064071780614]

In [ ]:
final_model = models[np.argmax(accuracies)] # take the model with highest accuracy
print(max(accuracies))

0.4733982054846455


In [ ]:
# optionally save the best performing model
final_model.save('attributes_regression')